In [6]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy as sp
import tmm

In [8]:
class Material:
    def __init__(self, path, name, wavelengths):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        nc = np.array(f.n+f.k*1j)  # complex refractive index
        self.name = name
        self.f = interp1d(wv_raw, nc, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.df = pd.Series(data=nc, index=wv_raw, name=name)
        # self.df = self.df.reindex(wavelengths)
        # self.df = self.df.interpolate('spline', order=3)
        self.interp(wavelengths)

    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        nc = self.f(wavelengths)
        self.df = pd.DataFrame(nc,wavelengths, [self.name])

In [4]:
class Model:
    def __init__(self):
        self.wavelength = np.arange(300, 1700, 1)
        self.increment = .2
        self.index_array = np.array([])
        self.materials = []
        self.mat_df = pd.DataFrame([], self.wavelength)
        self.data = {}
        self.thicknesses = [sp.inf, 0, sp.inf]
        self.layers = []

        self.add_material("TCO", './Materials/Semiconductor/TCO.csv')
        self.add_material("CdSe", './Materials/Semiconductor/CdSe.csv')
        self.add_material("CdTe", './Materials/Semiconductor/CdTe.csv')
        self.add_material("Air", './Materials/Semiconductor/Air.csv')
        self.add_material("BK7", './Materials/Dielectric/BK7.csv')
        self.add_material("SLG", './Materials/Dielectric/SLG.csv')
        self.add_material("SS TCO", './Materials/Dielectric/Sunnyside TCO.csv')
        self.add_material("SiO2", './Materials/Dielectric/SiO2-Jaw.csv')
        
    def add_material(self, film, path):
        if film not in self.mat_df:
            mat = Material(path, film, self.wavelength)
            self.materials.append(mat)
            self.mat_df = self.mat_df.join(mat.df)

    def set_wavelength(self, low, high, interval):
        self.wavelength = np.arange(low, high, interval)
        df = self.mat_df.reindex(self.wavelength)
        df = df.interpolate('spline', order=3)
        self.mat_df = df

    def better_bruggeman(self, n1, n2, percent_included):
        p = n1/n2
        b = .25*((3*percent_included-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*n1*n2
        return {"e": e, "n": e**0.5, 'conc': percent_included, "n1": n1, 'n2': n2}

    def brug_transform(self, df, layer, incl, percent):
        p = df[layer]/incl
        b = .25*((3*percent-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*df[layer]*incl
        n = e**.5
        df[layer] = n
    
    def run(self, wavelengths, void_percent):
        mat = self.mat_df.ix[wavelengths]
        mat = mat[self.layers]
        self.brug_transform(mat, self.layers[1], mat['Air'], void_percent)
        self.index_array = np.array(mat)
        theta0 = 45*sp.pi/180
        self.data = tmm.unpolarized_RT(self.index_array, self.thicknesses, theta0, wavelengths)
        
    def normalized(a, axis=-1, order=2):
        l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
        l2[l2==0] = 1
        return a / np.expand_dims(l2, axis)
    
    def get_R(self, wavelengths, thickness, void_percent):
        self.thicknesses[1] = thickness
        self.run(wavelengths, void_percent) 
        R = self.data['R']
        R -= min(R)
        R /= max(R)
        return R
        
    def RMSE(self, thickness, data):
        df = pd.DataFrame(data)
        self.get_R(thickness)
        model = pd.DataFrame(self.data['R'], index=self.wavelength)
        df = df.join(model, how='inner')
        n = len(df.index)
        return (sum((data-model)**2)/n)**0.5
    
    def norm(self, wavelength):
        df = self.df.ix[wavelength]
        df = df - df.min()
        df = df / df.max()
        return df
    
    def fit(self, wv, data):
        mod = lmfit.Model(self.get_R, ['wavelengths'], ['thickness','void_percent'])
        mod.set_param_hint('thickness', value = 120, min=50, max=250)
        mod.set_param_hint('void_percent', value = .15, min=0, max=1)
        
        weight = np.array(data.weight.ix[wv])
        R = data.series.ix[wv]
        result = mod.fit(R, wavelengths=wv, weights=weight)
        
        RMSE = (sp.sum(result.residual**2)/(result.residual.size-2))**0.5
        bf_values = result.best_values
        bf_str = 'thk: ' + str(round(bf_values['thickness'])) +", Void %: " + str(round(bf_values['void_percent']*100, 2))
        txt_spot = wv.min()-200 + (wv.max()-wv.min()) / 2

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.text(txt_spot, .9, "RMSE: "+str(round(RMSE, 3)))
        ax.text(txt_spot, .85, bf_str)
        result.plot_fit(yerr=np.zeros(len(data.series.index)), data_kws ={'marker':'+'})

        plt.show()
        print(result.fit_report())

In [9]:
model = Model()

In [26]:
def unpolarized_R(n_list, d_list, th_0, lam_vac):
    """
    Calculates reflected and transmitted power for unpolarized light.
    """

    s_data = tmm.coh_tmm('s',n_list, d_list, th_0, lam_vac)
    p_data = tmm.coh_tmm('p',n_list, d_list, th_0, lam_vac)
    R = (s_data['R'] + p_data['R']) / 2.
    return R

In [29]:
mat = model.mat_df[['Air', 'SiO2', 'TCO']]
d_list = [sp.inf, 120, sp.inf]
theta = 55*sp.pi/180
s = pd.Series(name='R')
for i in mat.index:
    index_array = np.array(mat.loc[i])
    r = unpolarized_R(index_array, d_list, theta, i)
    s = s.set_value(i, r)


In [30]:
from matplotlib import pyplot as plt
plt.plot(s.index, s)
plt.show()

In [28]:
mat = model.mat_df[['Air', 'SiO2', 'TCO']]
d_list = [sp.inf, 120, sp.inf]
theta = 55*sp.pi/180
mat.apply(unpolarized_R, axis=1, args=(d_list, theta, mat.index))

ValueError: ('This function is not vectorized; you need to run one calculation at a time (1 wavelength, 1 angle, etc.)', 'occurred at index 300')

In [62]:
data = Data('C:/WORK!!!!/VF088 - ARC Thickness/EKE-HC d.csv', 'Data', np.arange(300,1700), 'R')


In [44]:
z = 10
power = 3
deriv = 0

df = data.df
m = z*2+1

mirror = np.array(df.loc[df.index.min()+1:df.index.min()+1+z])[::-1]
pre = pd.Series(mirror, np.arange(df.index.min()-1-z, df.index.min()), name='Data')

mirror = np.array(df.loc[df.index.max()-z:df.index.max()-1])[::-1]
print(mirror)
app = pd.Series(mirror, np.arange(df.index.max()+1, df.index.max()+1+z), name='Data')
df = pd.concat([pre, df, app])

[ 0.62479731  0.62624661  0.62480601  0.62897326  0.6296635   0.62917771
  0.62809776  0.6267937   0.62725939  0.62835453]


331    0.678040
332    0.645101
333    0.652057
334    0.892480
335    0.835449
336    0.624655
337    0.583661
338    0.777643
339    0.857572
340    0.527690
341    0.581022
342    0.638365
343    0.581022
344    0.527690
345    0.857572
346    0.777643
347    0.583661
348    0.624655
349    0.835449
350    0.892480
351    0.652057
352    0.645101
353    0.678040
354    0.679424
355    0.702101
356    0.732835
357    0.735163
358    0.770052
359    0.729046
360    0.737625
         ...   
913    0.625646
914    0.625465
915    0.622660
916    0.622779
917    0.628249
918    0.630314
919    0.633398
920    0.629746
921    0.627813
922    0.628355
923    0.627259
924    0.626794
925    0.628098
926    0.629178
927    0.629664
928    0.628973
929    0.624806
930    0.626247
931    0.624797
932    0.619225
933    0.624797
934    0.626247
935    0.624806
936    0.628973
937    0.629664
938    0.629178
939    0.628098
940    0.626794
941    0.627259
942    0.628355
Name: Data, dtype: float

In [57]:
from matplotlib import pyplot as plt

In [66]:
d = 

In [ ]:
data = np.arange(1, 26)
index = np.arange(1, 6, .2).round(1)

x = pd.DataFrame(np.array([data, index.astype(int)]).T, columns=['data', 'index'])
x.mean(1)

In [ ]:
x = 3
y = -4

np.arctan(y/x)*180/np.pi

In [34]:
a = np.arange(6).reshape(2,3)
b = np.arange(6,12).reshape(2,3)
c = np.arange(12,18).reshape(2,3)
d = np.arange(18,24).reshape(2,3)
x = np.array([[a, b], [c, d]])

In [38]:
np.array([[a, b], [c, d]])

array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]],


       [[[12, 13],
         [14, 15]],

        [[16, 17],
         [18, 19]],

        [[20, 21],
         [22, 23]]]])

In [54]:
def pretend(np_array):
    return sum(np_array)*np_array.name

df = pd.DataFrame(np.arange(16).reshape(8,2))
answer = df.apply(pretend,axis=1)
answer

0      0
1      5
2     18
3     39
4     68
5    105
6    150
7    203
dtype: int64